In [1]:
import json
import nltk
import numpy as np
import random

In [2]:
data_file = open('D:/AI STUDY MATERIAL/info.json').read()
intents = json.loads(data_file)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'no_answer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'identity',
   'patterns': ['Who are you?', 'What is your name?', 'What 

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
for i in intents['intents']:
    for j in i['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(j)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, i['tag']))
        # add to our classes list
        if i['tag'] not in classes:
            classes.append(i['tag'])

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [5]:
ignore_words=['?','!']
words= [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words= sorted(list(set(words)))

In [6]:
classes=sorted(list(set(classes)))
classes

['coursedetail',
 'coursefee',
 'datascience',
 'dsgrowth',
 'goodbye',
 'greeting',
 'identity',
 'options',
 'thanks']

In [7]:
print (len(documents), "documents")

35 documents


In [8]:
print (len(classes), "classes", classes)

9 classes ['coursedetail', 'coursefee', 'datascience', 'dsgrowth', 'goodbye', 'greeting', 'identity', 'options', 'thanks']


In [9]:
print (len(words), "unique lemmatized words", words)

57 unique lemmatized words ["'s", ',', 'about', 'anyone', 'are', 'awesome', 'bye', 'can', 'chatting', 'course', 'data', 'day', 'do', 'fee', 'for', 'good', 'goodbye', 'growth', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'know', 'later', 'learning', 'me', 'much', 'name', 'need', 'next', 'nice', 'of', 'offered', 'pay', 'science', 'scope', 'see', 'support', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'use', 'what', 'who', 'you', 'your']


In [10]:
import pickle
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [11]:
#training_data
training=[]

output_empty=[0]*len(classes)

In [12]:
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words= [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])

In [13]:
training1=[]

for doc in documents:
    bag1=[]
    a1=doc[0]
    a2= [lemmatizer.lemmatize(i.lower()) for i in a1]
    #print(a2)
    
    for w in words:
        bag1.append(1) if w in a2 else bag1.append(0)
    #print(bag)
    output_row1 = list(output_empty)
    output_row1[classes.index(doc[1])]=1
    #print(output_row)
    
    training1.append([bag1,output_row1])


In [14]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD


Using TensorFlow backend.


In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('model created')

Epoch 1/200
35/35 [==============================] - 1s 22ms/step - loss: 2.2729 - accuracy: 0.1143
Epoch 2/200
35/35 [==============================] - 0s 457us/step - loss: 2.1691 - accuracy: 0.0857
Epoch 3/200
35/35 [==============================] - 0s 457us/step - loss: 2.1027 - accuracy: 0.2286
Epoch 4/200
35/35 [==============================] - 0s 457us/step - loss: 2.0308 - accuracy: 0.3143
Epoch 5/200
35/35 [==============================] - 0s 229us/step - loss: 1.9273 - accuracy: 0.4000
Epoch 6/200
35/35 [==============================] - 0s 229us/step - loss: 1.8704 - accuracy: 0.3429
Epoch 7/200
35/35 [==============================] - 0s 457us/step - loss: 1.7668 - accuracy: 0.4286
Epoch 8/200
35/35 [==============================] - 0s 457us/step - loss: 1.7263 - accuracy: 0.4000
Epoch 9/200
35/35 [==============================] - 0s 457us/step - loss: 1.7262 - accuracy: 0.4857
Epoch 10/200
35/35 [==============================] - 0s 686us/step - loss: 1.6882 - accurac

35/35 [==============================] - 0s 686us/step - loss: 0.0855 - accuracy: 1.0000
Epoch 81/200
35/35 [==============================] - 0s 685us/step - loss: 0.0651 - accuracy: 1.0000
Epoch 82/200
35/35 [==============================] - 0s 686us/step - loss: 0.0628 - accuracy: 0.9714
Epoch 83/200
35/35 [==============================] - 0s 686us/step - loss: 0.0734 - accuracy: 1.0000
Epoch 84/200
35/35 [==============================] - 0s 686us/step - loss: 0.1214 - accuracy: 0.9714
Epoch 85/200
35/35 [==============================] - 0s 686us/step - loss: 0.0938 - accuracy: 1.0000
Epoch 86/200
35/35 [==============================] - 0s 686us/step - loss: 0.0412 - accuracy: 1.0000
Epoch 87/200
35/35 [==============================] - 0s 686us/step - loss: 0.0893 - accuracy: 1.0000
Epoch 88/200
35/35 [==============================] - 0s 686us/step - loss: 0.0475 - accuracy: 1.0000
Epoch 89/200
35/35 [==============================] - 0s 685us/step - loss: 0.1260 - accuracy: 

35/35 [==============================] - 0s 457us/step - loss: 0.0939 - accuracy: 0.9714
Epoch 160/200
35/35 [==============================] - 0s 686us/step - loss: 0.0604 - accuracy: 1.0000
Epoch 161/200
35/35 [==============================] - 0s 686us/step - loss: 0.0404 - accuracy: 0.9714
Epoch 162/200
35/35 [==============================] - 0s 457us/step - loss: 0.0327 - accuracy: 1.0000
Epoch 163/200
35/35 [==============================] - 0s 457us/step - loss: 0.0184 - accuracy: 1.0000
Epoch 164/200
35/35 [==============================] - 0s 457us/step - loss: 0.0502 - accuracy: 1.0000
Epoch 165/200
35/35 [==============================] - 0s 457us/step - loss: 0.1225 - accuracy: 0.9429
Epoch 166/200
35/35 [==============================] - 0s 457us/step - loss: 0.0188 - accuracy: 1.0000
Epoch 167/200
35/35 [==============================] - 0s 457us/step - loss: 0.0497 - accuracy: 0.9714
Epoch 168/200
35/35 [==============================] - 0s 457us/step - loss: 0.0515 - a